In [1]:
# import python library
import tensorflow as tf
import keras
import numpy as np
import cv2 
import matplotlib.pyplot as plt
# configure and test
tf.test.gpu_device_name()
% matplotlib inline

Using TensorFlow backend.


In [23]:
a = tf.constant([[1, 2,3],[1, 2,3]], dtype = tf.float32)
a.get_shape().as_list()[1]

3

In [35]:
# define the neural network architecture 
# define needed variables during training 
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization
def _variable_summaries(var):
    """
    Attach a lot of summaries to a Tensor (for TensorBoard visualization).
    """
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
    
class model(object):
    # class level attributes, used for 
    number = 0
    def __init__(self, name, input_size, class_num):
        model.name = name
        w, h, c = input_size
        with tf.name_scope('input'):
            self.x  = tf.placeholder(tf.float32, shape = [None, w, h, c], name = 'input-x')
            self.y  = tf.placeholder(tf.float32, shape = [None, class_num], name ='input-y') 
    # finally build our model according to our own need
    def builder(self):
        self.conv1 = self._conv_layer(self.x,     [5, 5, 3, 32], 'conv1')
        self.conv2 = self._conv_layer(self.conv1, [3, 3, 32, 64], 'conv2')
        self.pool1 = self._max_pool22(self.conv2)
        # 
        self.ffn1  = self._ffnet_module(self.pool1,f = [3, 5], filters = [64, 64, 64, 64] , stage = 1, block = 'a') 
        self.fc1   = self._fc_layer(tf.reshape(ffnet, [-1, 12*12*128]), 12*12*128, 256, 'fc1' )
        self.fc2   = self._fc_layer(self.fc1, 256, 10, 'fc2')
        self.y_    = tf.nn.softmax(self.fc2, 2)
    # class methods to construct conv-layer, generating summary autotically 
    # with default stride 1, and relu activations
    def _conv_layer(self, X, shape, layer_name, padding='SAME'):
        with tf.name_scope(layer_name):
            with tf.name_scope('W'):
                weight = self.get_filters(shape)
                _variable_summaries(weight)
            with tf.name_scope('b'):
                bias = self.get_bias(shape[3])
                print(shape[3])
                _variable_summaries(bias)
            with tf.name_scope('z'):
                preactivations = tf.nn.conv2d(X, weight, strides=[1, 1, 1, 1], padding=padding) + bias
                _variable_summaries(preactivations)
            with tf.name_scope('A'):
                activations = tf.nn.relu(preactivations)
                _variable_summaries(activations)
        return activations
    
    # ex: prob = tf.constant(0.5, dtype=tf.float32)
    def _drop_out(self, X, prob, layer_name):
        with tf.name_scope(layer_name+'Dropout'):
            return tf.nn.dropout(X, prob)
    def _max_pool22(self, X, padding = 'SAME'):
        return tf.nn.max_pool(x, strides= [1, 2, 2, 1], ksize=[1, 2, 2, 1], padding= padding)
           
    def _fc_layer(self, X, input_dim, output_dim, layer_name, act=tf.nn.relu):
        with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
            with tf.name_scope('W'):
                weights = self.get_weights([input_dim, output_dim])
                variable_summaries(weights)
            with tf.name_scope('b'):
                biases = self.get_bias([output_dim])
                variable_summaries(biases)
            with tf.name_scope('z'):
                preactivate = tf.matmul(input_tensor, weights) + biases
                tf.summary.histogram('pre_activations', preactivate)
            activations = act(preactivate, name='activation')
            tf.summary.histogram('activations', activations)
        return activations

    def _ffnet_module(self, X, f, filters , stage, block):
        """
        Implementation of the figure above, with 3 standard 3*3*64 module for the general module, and one fast-forwarding path
        Arguments: 
        X--      the input tensor with shape (n_H, n_W, n_C)
        f--      filter kernel size
        filters: number of filters in each layer
        stage  : name of stage 
        block  : string/character, used to name the layers, depending on 

        Returns:
        X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
        """
        conv_name_base = 'ffnet' + str(stage) + block + 'conv_branch'
        bn_name_base   = 'bn' + str(stage) + block + 'bn_branch'
        fast_fwd_base  = 'ffnet' + str(stage) + block + 'fast_branch' 

        # Retrieve filters 
        F1, F2, F3, F4 = filters
        f1, f2     = f
        m = X.get_shape().as_list()[3]

        # Save the input value
        X_fast = X

        ####### Main Path ######
        X = self._conv_layer(X, [f1, f1, m, F1],  layer_name = conv_name_base + '1', padding = 'VALID')
        setattr(self, 'ffnet'+str(stage)+'_1', X)
        X = self._conv_layer(X, [f1, f1, F1, F2], layer_name = conv_name_base + '2', padding = 'VALID')
        setattr(self, 'ffnet'+str(stage)+'_2', X)
        X = self._conv_layer(X, [f1, f1, F2, F3], layer_name = conv_name_base + '3')
        ####### fast forward ###### 
        X_fast = self._conv_layer(X_fast, [f2, f2, m, F4], layer_name = fast_fwd_base + '1')
        setattr(self, 'ffnet'+str(stage)+'_4', X_fast)
        result = tf.concat([X, X_fast], 3) 
        print(X_fast)
        print(X)
        print(result)
        ###### Final step: concatation #######

        return result
        
    def get_filters(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return(tf.Variable(initial))
    
    def get_weights(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return(tf.Variable(initial))
    
    def get_bias(self, output_dim):
        initial = tf.constant(0.1, shape = output_dim)
        return tf.Variable(initial)

        
        
        
        
        
    

    

In [36]:
#module test
ffnet_s1 = model('ffnet', [32, 32, 3], 2)
ffnet_s1.builder()

TypeError: 'int' object is not iterable

In [ ]:
# Data import with Dataset
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)  # Parse the record into tensors.
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()

# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.

# Initialize `iterator` with training data.
training_filenames = ["train-00000-of-00001"]
train = sess.run(iterator.initializer, feed_dict={filenames: training_filenames})

# Initialize `iterator` with validation data.
# validation_filenames = ["validation-00000-of-00001"]
# val = sess.run(iterator.initializer, feed_dict={filenames: validation_filenames})

In [6]:
################# start to build up our model !###############
x = tf.placeholder(tf.float32, shape = [None, 32, 32, 3])
y_ = tf.placeholder(tf.float32, shape = [None, 2])
# the first convolutional layer 
conv_1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

#x_image = tf.reshape(x, [-1, 32, 32, 3])
h_conv1 = tf.nn.relu(conv2d(x, conv_1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
# the second convolutional layer+relu+ 
conv_2 =  weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, conv_2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
ffnet = ffnet_module(h_pool1, f = [3, 5], filters = [64, 64, 64] , stage = 1, block = 'a', s = 1 )
# dense connected layer 1
W_fc1 = weight_variable([12*12*128, 256])
b_fc1 = bias_variable([256])
hpool_flat = tf.reshape(ffnet, [-1, 12*12*128])
h_fc1 = tf.nn.relu(tf.matmul(hpool_flat, W_fc1) + b_fc1)
# dense connected layer 2, etc
W_fc2 = weight_variable([256, 10])
b_fc2 = bias_variable([10])
# dense connected layer 3, etc
W_fc3 = weight_variable([10, 2])
b_fc3 = bias_variable([2])
# Dropout design
prob = tf.placeholder(tf.float32)
h_fc1dropout = tf.nn.relu(tf.nn.dropout(h_fc1, prob))
h_fc2= tf.matmul(h_fc1dropout, W_fc2) + b_fc2
# output channel [node1, node2]
y_conv = tf.matmul(h_fc2, W_fc3) + b_fc3


Tensor("Relu_5:0", shape=(?, 12, 12, 64), dtype=float32)
Tensor("Relu_4:0", shape=(?, 12, 12, 64), dtype=float32)
Tensor("concat:0", shape=(?, 12, 12, 128), dtype=float32)


In [ ]:
# import data in the format of tf.
height=tf.constant(32, dtype=tf.int32)
width =tf.constant(32, dtype=tf.int32)
def read_and_decode(filename, batch_size, num_epochs):
    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer([filename], num_epochs=num_epochs)
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
            'image/height': tf.FixedLenFeature([], tf.int64),
            'image/width': tf.FixedLenFeature([], tf.int64),
            'image/colorspace': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/channels':  tf.FixedLenFeature([], tf.int64),            
            'image/class/label': tf.FixedLenFeature([],tf.int64),
            'image/class/text': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/format': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/filename': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/encoded': tf.FixedLenFeature([], dtype=tf.string, default_value='')
      })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
#     image = tf.decode_raw(features['image_raw'], tf.uint8)
#     image.set_shape([mnist.IMAGE_PIXELS])
    label = features['image/class/label']
    image_buffer = features['image/encoded']

    # Decode the jpeg
    with tf.name_scope('decode_jpeg',[image_buffer], None):
        # decode
        image = tf.image.decode_jpeg(image_buffer, channels=3)
        
        # resize
        image = tf.image.resize_images(image, [height, width])
    
        # and convert to single precision data type
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)


    # OPTIONAL: Could reshape into a 28x28 image and apply distortions
    # here.  Since we are not applying any distortions in this
    # example, and the next step expects the image to be flattened
    # into a vector, we don't bother.

    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    print(image.shape)
    #image=tf.reshape(image,[height*width*3])
    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(label, tf.int32)  
#     images, sparse_labels = tf.train.shuffle_batch(
#         [image, label], batch_size=batch_size, num_threads=2,
#         capacity=1000 + 3 * batch_size,
#         # Ensures a minimum amount of shuffling of examples.
#         min_after_dequeue=1000)
    return image, label
images, labels = read_and_decode("train-00000-of-00001", 1, 10)
vimages, vlabels = read_and_decode("validation-00000-of-00001", 1, 10)
# training examples 
imageBatch, labelBatch = tf.train.shuffle_batch(
    [images, labels], batch_size=100,
    capacity=2000,
    min_after_dequeue=1000)
# and similarly for the validation data 
vimageBatch, vlabelBatch = tf.train.shuffle_batch(
    [vimages, vlabels], batch_size=1,
    capacity=2000,
    min_after_dequeue=1000)
# # interactive session allows inteleaving of building and running steps
# sess = tf.InteractiveSession()
# batch_size =  100
# dataset_train = dataset_train.batch(batch_size)
# iterator = dataset_train.make_one_shot_iterator()
# batch_features, batch_labels = iterator.get_next()
sess = tf.InteractiveSession()
print(imageBatch.eval())

(32, 32, 3)
(32, 32, 3)


In [ ]:
# explore ways to iomport data: to import data in an iterative style, while aslo assign data and labels dynamically
######1. Create Dataset object #######
# function for image reading
import cv2
def read_patch_cv2(filename, label):
    image_decoded = cv2.imread(filename, cv2.IMREAD_COLOR)
    print('here I come')
    return image_decoded, label

def _parse_function(filename, label):
    image_string = tf.read_file(filename)#labels are not put here 
    image = tf.image.decode_image(image_string, channels =  3)
    print('Transforming')
    return image, label

def imgs_input_fn(filenames, labels = None):
    # create labels if there is no source
    if labels is None:
        labels = [0]*len(filenames)
    labels = np.array(labels)
    # to avoid 
    if len(labels) == 1:
        np.expand_dims(labels, axis = 1)
    # convert array/list into tensorflow constants
    filenames = tf.constant(filenames)
    labels = tf.constant(labels)
    #abels = tf.cast(labels, tf.float32)
    # apply standard tf function mapping images data to dataset
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(_parse_function)
    return dataset

def read_txt(filepath, prefix):
    filenames = []
    labels= []
    # read from text files 
    with open(filepath, 'r') as f:
        num = 0
        for line in f:
            num +=1 
            line = line.strip()
            if num < 1:
                print('%s'  %(line[:-2]) )
                print(line[-1])
            filenames.append(prefix+line[:-2])
            labels.append(line[-1])
    print(num)
    return filenames, labels
    
#print(filenames)
filepath_train = '/home/dragonx/Documents/text_detect/dataC/COCO-Text-Patch/train.txt'
filepath_val = '/home/dragonx/Documents/text_detect/dataA/COCO-Text-Patch/val.txt'
filenames_train, labels_train = read_txt(filepath_train, './dataC/COCO-Text-Patch/images/')
filenames_val, labels_val = read_txt(filepath_val, './dataA/COCO-Text-Patch/images/')
dataset_train = imgs_input_fn(filenames_train, labels_train)
dataset_val   = imgs_input_fn(filenames_val, labels_val)
######2. batch_read() ####### 
#either from file directory, or by the txt file with filenames  

######3. Create tfRecord Object #######

#(recommended)
######4. Read from npy format(one by one or in total)  #######

In [ ]:
# Data import with Dataset
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)  # Parse the record into tensors.
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()

# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.

# Initialize `iterator` with training data.
training_filenames = ["train-00000-of-00001"]
train = sess.run(iterator.initializer, feed_dict={filenames: training_filenames})

# Initialize `iterator` with validation data.
# validation_filenames = ["validation-00000-of-00001"]
# val = sess.run(iterator.initializer, feed_dict={filenames: validation_filenames})

In [7]:
# make sure the directory is correct
!pwd
def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    
    ### START CODE HERE ###
    
    # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    C = tf.constant(C, name='C')
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = tf.one_hot(indices=labels, depth=C, axis=0)
    
    # Create the session (approx. 1 line)
    sess = tf.Session()
    
    # Run the session (approx. 1 line)
    one_hot = sess.run(one_hot_matrix)
    
    # Close the session (approx. 1 line). See method 1 above.
    sess.close()
    
    ### END CODE HERE ###
    
    return one_hot

/home/dragonx/Documents/text_detect


In [8]:
# self defined data import as numpy array
import os
import cv2
train_dir = './dataC/COCO-Text-Patch/images/'
val_dir =   './dataA/COCO-Text-Patch/images/'
classes = ['txt', 'nontxt']
fname_tr   = []
labels_tr  = []
fname_val  = []
labels_val = []
for i, c in enumerate(classes):
    buffer = os.listdir(train_dir+c)
    buffer = [train_dir+c+'/'+x for x in buffer]
    fname_tr.extend(buffer[:])
    labels_tr.extend([i]*len(buffer))
    # the same for val
    buffer = os.listdir(val_dir+c)
    buffer = [val_dir+c+'/'+x for x in buffer]
    fname_val.extend(buffer[:])
    labels_val.extend([i]*len(buffer))

print(len(fname_tr))
print(len(fname_val))
# print(fname_tr[0:100])
# check if numbers are correct
assert len(fname_tr) == len(labels_tr)
index_tr = np.arange(0, len(labels_tr))  
index_val= np.arange(0, len(labels_val))  
def imread_batch(filenames, labels):
    train_data = np.zeros([len(filenames), 32, 32, 3])
    for i, f in enumerate(filenames):
        #print(f)
        buffer = cv2.imread(f)
        train_data[i, :,:,:] = buffer[:,:,:]
        #print(buffer.shape)
    labels = one_hot_matrix(labels, 2).T
    
    return train_data, labels

#data_tr, labels_tr = imread_batch(fname_tr[0:100], labels_tr[0:100])
#data_val,labels_val = imread_batch(fname_val[0:100], labels_val[0:100])
# data_tr, labels_tr = imread_batch(fname_tr(index_tr[0:100]) , labels_tr[0:100])
print(np.take(fname_tr, index_tr[1:10], axis = 0))          

108012
122324
['./dataC/COCO-Text-Patch/images/txt/txt_615639.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_660131.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_235518.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_734094.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_336638.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_374506.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_127162.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_713484.jpg'
 './dataC/COCO-Text-Patch/images/txt/txt_43367.jpg']


In [9]:
# input data produce
data_batch, labels_batch = imread_batch(fname_tr[1:100], labels_tr[1:100]); 
print(data_batch.shape)
print(labels_batch.shape)

(99, 32, 32, 3)
(99, 2)


In [10]:
saver = tf.train.Saver()
# OOD programming
def _create_summaries(loss, accuracy):
    with tf.name_scope("summaries"):
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.scalar("histogram loss", loss)
        # merge together
        summary_op = tf.summary.merge_all()
    return summary_op

In [18]:
############ for training detail ##############
# x, y_: input labels
# y_conv: convolutional layers output
from random  import shuffle
import time
# loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y_conv))
# optimizer and learning rate
train_step   = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# how to decide the prediction and accuracy
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# train process 
batch_size    = 100
epoch_num     = 20
train_record  = []
global_step   = tf.Variable(10, dtype=tf.int32, trainable=False, name = 'global_step')
check_summary = _create_summaries(cross_entropy, accuracy)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('./checkpoint/check_summary', sess.graph)
    # num_tr = np.floor(len(labels_tr)/batch_size)
    num_val = np.floor(len(labels_val)/batch_size)
    num_tr = 200
    for i in range(epoch_num):
        start_time = time.time()
        print('epoch %d'%(i) )
        # random batch in every iteration
        shuffle(index_tr)
        for j in range(np.int64(num_tr)):
            # data preparation
            sub_time = time.time()
            filenames_batch = np.take(fname_tr, index_tr[batch_size*(j) :min(batch_size*(j+1), len(labels_tr))], axis = 0)
            labels_batch    = np.take(labels_tr, index_tr[batch_size*(j):min(batch_size*(j+1),len(labels_tr))] ,axis = 0)
            data_batch, labels_batch = imread_batch(filenames_batch, labels_batch);#print(data_batch.shape)
            sub_time1 = time.time()
            #print('loading time on %d images is %f'%(batch_size, sub_time1 -sub_time))   
            # train the network from scratch using GPU
            train_step.run(feed_dict = {x: data_batch, y_ :labels_batch, prob:0.5})
            summary = sess.run(check_summary, feed_dict = {x: data_batch, y_ :labels_batch, prob:0.5})
            summary_writer.add_summary(summary, i*num_tr+j)
            epoch_check = 5
#           if (j+1)%5 == 0:
#               saver.save(sess, 'checkpoint/FFnet_Classify_V0', global_step = global_step )
            if j%50 == 0:
                print('load & training time on %d images is %f'%(batch_size, time.time()-sub_time))
        # model evaluation with val dataset
        train_accuracy = accuracy.eval(feed_dict = {x: data_batch, y_ : labels_batch, prob: 1})
        train_record.append(train_accuracy)
        print('current accuracy on %d images is %f'%(batch_size, train_accuracy))
        # training time evaluation with timer
        # provide checkpoints to save models sequentially
        epoch_check = 5
        if (i+1)%5 == 0:
            saver.save(sess, 'checkpoint/FFnet_Classify_V0', global_step = tf.convert_to_tensor(epoch_check*num_tr))
            
            
        # data visualization & model summary  
        
#         if i % 10 == 0:
#             train_accuracy = accuracy.eval(feed_dict = {x: , y_ : labelBatch, prob: 1})
#             print('step %d, training accuracy %g' %(i, train_accuracy))
#         filenames_batch = np.take(fname_tr, index_tr[batch_size*(j):batch_size*(j+1)], axis = 0)
#         labels_batch    = np.take(labels_tr, index_tr[batch_size*(i):batch_size*(i+1)], axis = 0)
#         data_batch, labels_batch = imread_batch(filenames_batch, labels_batch)
#         print('test accuracy %g' %accuracy.eval(feed_dict = {x: vimageBatch, y_: vimageBatch, prob : 1.0}))

INFO:tensorflow:Summary name histogram loss is illegal; using histogram_loss instead.
epoch 0
load & training time on 100 images is 0.634101
load & training time on 100 images is 0.546075
load & training time on 100 images is 0.986707
load & training time on 100 images is 0.548373
current accuracy on 100 images is 0.660000
epoch 1
load & training time on 100 images is 0.566722
load & training time on 100 images is 1.513245
load & training time on 100 images is 1.101350
load & training time on 100 images is 1.203141
current accuracy on 100 images is 0.680000
epoch 2
load & training time on 100 images is 0.561458
load & training time on 100 images is 1.285371
load & training time on 100 images is 0.702812
load & training time on 100 images is 0.713212
current accuracy on 100 images is 0.710000
epoch 3
load & training time on 100 images is 0.945154
load & training time on 100 images is 0.967590
load & training time on 100 images is 0.705250
load & training time on 100 images is 0.666004
c

In [20]:
train_record

[0.66000003,
 0.68000001,
 0.70999998,
 0.72000003,
 0.79000002,
 0.75,
 0.77999997,
 0.69,
 0.75999999,
 0.70999998,
 0.75,
 0.75,
 0.79000002,
 0.83999997,
 0.81999999,
 0.80000001,
 0.73000002,
 0.82999998,
 0.85000002,
 0.81]

In [32]:
sess = tf.Session()
saver = tf.train.import_meta_graph('checkpoint/FFnet_Classify_V0-0.meta')
saver.restore(sess,tf.train.latest_checkpoint('./checkpoint/'))
#saver.restore(sess, 'checkpoint/FFnet_Classify_')

OSError: File checkpoint/FFnet_Classify_V0-0.meta does not exist.

In [31]:
# not ood style of summary
!pwd

/home/dragonx/Documents/text_detect
